In [1]:
!pip install langchain
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### Requirements

In [2]:

from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage, # AI Input
    HumanMessage, # user input
    SystemMessage # sets the tone of the conversation
)
from langchain.evaluation.qa import QAEvalChain
import openai
from langchain.llms import OpenAI

### Setting up the Chat Model



In [3]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-API-KEY"

### Few shot prompt


In [6]:

few_shot_examples = [
  {
    "question": "Hello, can you please help me solve this problem. John has 2 houses. Each house has 3 bedrooms and there are 2 windows in each bedroom. \
    Each house has 1 kitchen with 2 windows. Also, each house has 5 windows that are not in the bedrooms or kitchens. \
    How many windows are there in John's houses?",
    "answer":
"""
Clues:
1. Find the number of bedroom windows, kitchen windows, and other windows separately.
2. Add them together to find the total number of windows at each house.
3. Find the total number of windows for all the houses.

Steps:
- Each house has 3 bedrooms with 2 windows each, so that's 3 x 2 = 6 windows per house.
- Each house also has 1 kitchen with 2 windows, so that's 2 x 1 = 2 windows per house.
- Each house has 5 windows that are not in the bedrooms or kitchens, so that's 5 x 1 = 5 windows per house.
- In total, each house has 6 + 2 + 5 = 13 windows.
- Since John has 2 houses, he has a total of 2 x 13 = 26 windows.

Final Answer: 26
"""
  },
  {
    "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
    "answer":
"""
Clues:
1. Start with the total number of chocolates they had.
2. Subtract the number of chocolates they ate.

Steps:
- Originally, Leah had 32 chocolates. Her sister had 42.
- So in total they had 32 + 42 = 74.
- After eating 35, they had 74 - 35 = 39.

Final Answer: 39
"""
  },
  {
    "question": "Solve the system of equations:\
    1. 2x + 3y = 8 \ 2. x - y = 1",
    "answer":
"""
Clues:
1. Use methods like substitution or elimination.
2. Express x in terms of y from one equation.
3. Plug in the value of x in the other equation to get the value of y.

Steps:
- From equation 2, express x as x = 1+y. Substitute this in equation 1. This yields, 2(1 + y) + 3y = 8 which simplifies to 5y = 6, hence y = 1.2
- Substituting y = 1.2 in equation 2, we get x = 2.2.

Final Answer: x = 2.2, y = 1.2
"""
  },
  {
    "question": "Solve the equation x^2 - 4x + 4 = 0",
    "answer":
"""
Clues:
1. Factorize the quadratic equation or use the quadratic formula.

Steps:
- This quadratic equation can be factored as (x - 2)(x - 2) = 0.
- The solutions are x = 2 (with multiplicity 2).

Final Answer: x = 2 (with multiplicity 2)
"""
  }
]

system_message = """You are a helpful AI Elementary Math tutor that teaches elementary school kids how to solve math problems.\
 If you receive a question, do not give away the answer right away. Give the students clues first and \
 then eventually work out the explanation with the answer.

Please output in the following format:

############# OUTPUT ############
Question: "Question here"
Clues: "Clues from the question"
Steps: "Steps to get to the final answer"
Answer: "Final answer here"
############ OUTPUT #############

Some examples of answers corresponding to elementary math questions are:

{few_shot_examples}

Think step by step

Question: {input}"""


### Helper functions


In [7]:
def generate_prompt(few_shot_examples=few_shot_examples, system_message=system_message):
    """
    Given few shot examples, and a system message to set the tone of the conversation,
    generate a prompt for the AI tutor

      Args:
          few_shot_examples (list, optional): examples using few shot prompting
          system_message (str, optional): A system message providing guidance to
              the AI math tutor.
              Defaults to system_message defined elsewhere.

      Returns:
          list: A list of messages including the system message and user-generated examples
    """

    messages = [SystemMessage(content=system_message)]
    for example in few_shot_examples:
        user_message = example["question"]
        ai_message = example["answer"]
        messages.append(HumanMessage(content=user_message))
        messages.append(AIMessage(content=ai_message))
    return messages


def prompt_response(question, few_shot_examples=few_shot_examples, system_message=system_message):
    """
    Ask a question to the AI tutor

        Args:
            question (string): An elementary math question
            few_shot_examples (list, optional): examples used to train the tutor
            system_message (str, optional): System message used to train the tutor

        Returns:
            AI Tutor's response to the question

    """
    chat = ChatOpenAI(temperature=0)
    messages = generate_prompt(few_shot_examples, system_message)
    messages.append(HumanMessage(content=question))
    return chat(messages).content


# Interactive Session
def interactive_session():
    """
    Start an interactive session with the tutor
    """
    chat = ChatOpenAI(temperature=0,
                    max_tokens=1000)
    messages = generate_prompt()
    while True:
      user_message = input("You: ")
      messages.append(HumanMessage(content=user_message))
      if user_message.lower() == "quit":
        break
      response = chat(messages)
      messages.append(response)

      print("AI:", response.content)


# Evaluate Math Tutor
def evaluate_math_tutor(questions_df):
    """
    Function to evaluate the answers generated by the tutor

      Args:
          questions_df (pandas.DataFrame): DataFrame containing the questions
                                          and correct answers. Each row in the
                                          DataFrame should represent one
                                          question and corresponding answer.

      Returns:
          list: Returns a list of dictionaries containing the results of the
                evaluation. Each dictionary contains the question, answer,
                and result of evaluation (e.g. 'correct', 'incorrect').
    """
    predictions = []

    for n in range(len(questions_df)):
        question = questions_df[n]['question']
        answer = prompt_response(question)
        predictions.append({'text': answer})
    llm = OpenAI(temperature=0)

    eval_chain = QAEvalChain.from_llm(llm)
    graded_outputs = eval_chain.evaluate(questions_df, predictions, question_key='question',
                                         answer_key='answer', prediction_key='text')

    return graded_outputs

# Analyze results
def analyze_results(results):
    """
    Computes and returns the accuracy of AI tutor based on the evaluation results.

    Args:
        results (list): List of dictionaries with keys 'results' denoting the correctness of the response.

    Returns:
        str: A string expressing the AI tutor's accuracy as a percentage.
    """

    total, correct, idx=0,0,0
    for v in results: # loop to compute accuracy
        total+=1; idx+=1
        if v['results'].strip()=='CORRECT':
            correct+=1

    return f'the model prediction accuracy is {correct/total*100} percent'


### Questions


In [13]:
math_questions = [
    {
        "question": "Simplify: (3x^2 y^3)(2x^3 y^2)",
        "answer": "6x^5 y^5"
    },
    {
        "question": "What is the value of √(169) + | -8 |?",
        "answer": "21"
    },
    {
        "question": "If a triangle has angles measuring 30°, 60°, and 90°, what is the length of the hypotenuse if one of the legs is 4 units long?",
        "answer": "8 units"
    },
    {
        "question": "Solve the equation: 3x + 7 = 2x - 4",
        "answer": "-11"
    },
    {
        "question": "Find the area of a circle with a radius of 5 units (use π ≈ 3.14).",
        "answer": "78.5 square units"
    },
    {
        "question": "Simplify: 5!(3!)",
        "answer": "720"
    },
    {
        "question": "Al fills an ice cream cone to the top edge. The diameter of the ice cream cone is 2 inches and its height is 6 inches. \
        Don fills a bowl (V = 5 cubic inches) with ice cream. Al has how much more ice cream than Don?",
        "answer": "1.28in^3"
    },
    {
        "question": "If log2(x) + log2(x-3) = 3, what is the value of x?",
        "answer": "4 or -2"
    },
    {
        "question": "A box contains 12 red, 8 blue, and 6 green balls. What is the probability of drawing a red ball?",
        "answer": "12/26"
    },
    {
        "question": "Solve for x: 2^(x+1) - 2^x = 16",
        "answer": "4"
    }
]

In [9]:
question = math_questions[-2]['question']
answer = prompt_response(question)
print(answer)

Clues:
1. Probability is calculated by dividing the number of favorable outcomes by the total number of possible outcomes.
2. In this case, the favorable outcome is drawing a red ball, and the total number of possible outcomes is the total number of balls in the box.

Steps:
- The total number of balls in the box is 12 + 8 + 6 = 26.
- The number of favorable outcomes (drawing a red ball) is 12.
- Therefore, the probability of drawing a red ball is 12/26.

Final Answer: The probability of drawing a red ball is 12/26.


### Evaluating the Model

In [14]:
results = evaluate_math_tutor(math_questions)

In [15]:
results

[{'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' INCORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'},
 {'results': ' CORRECT'}]

In [16]:
analyze_results(results)

'the model prediction accuracy is 90.0 percent'

### Evaluating the incorrect question

In [17]:
question = math_questions[-4]['question']
answer = prompt_response(question)
print(answer)


Clues:
1. Find the volume of the ice cream cone.
2. Compare the volume of the ice cream cone with the volume of the bowl.

Steps:
- The volume of a cone can be calculated using the formula V = (1/3) * π * r^2 * h, where r is the radius and h is the height.
- The radius of the ice cream cone is half of the diameter, so it is 2/2 = 1 inch.
- Plugging in the values, we get V = (1/3) * π * 1^2 * 6 = 2π cubic inches.
- Don's bowl has a volume of 5 cubic inches.
- Al has 2π - 5 cubic inches more ice cream than Don.

Final Answer: Al has 2π - 5 cubic inches more ice cream than Don.


**Even though the clues, steps and calculations is correct, the model struggles with getting to the right "answer".**

### Interactive Session


In [ ]:
interactive_session()

You: Hi can you help me with math questions?
AI: Of course! I'd be happy to help you with your math questions. Please go ahead and ask your question.
You: Suppose three days ago was Tuesday. What day of the week will it be 90 days from today?
AI: Clues:
1. Determine the number of days between the given day and the desired day.
2. Divide the number of days by 7 to find the number of weeks.
3. Determine the remainder to find the day of the week.

Steps:
- Three days ago was Tuesday, so we can start counting from Tuesday.
- We want to find the day of the week 90 days from today.
- Since there are 7 days in a week, we can divide 90 by 7 to find the number of weeks. 90 ÷ 7 = 12 remainder 6.
- This means that after 12 weeks, we will have 6 additional days.
- Starting from Tuesday, we count 6 days ahead: Wednesday, Thursday, Friday, Saturday, Sunday, and Monday.

Final Answer: 90 days from today will be Monday.
You: But the correct answer is Monday, you are not counting from the correct day. 

**We notice that even though the AI Tutor is receptive to feedback, it has no memory so it would forget the calibration it performed after prompting it to the correct answer and hence would give the wrong answer after asking the same question again.**



Go ahead and test the AI Tutor's capabilities with your own questions by running the following cell!

In [ ]:
interactive_session()